```
Eight Puzzle problem.  For example:

1 4 5                 1 2 3
2 3 7           =>    4 5 6
  6 7                 7 8   
```

**Lab Outline**

1.  Review search framework to see what components it requires

1.  Decide on our board representation (subclass of WorldState)
  1.  What does it have to store
  1.  What does it have to implement
  1.  Other considerations
    1.  Equality check to avoid repeating duplicated states
1. Decide on our problem representation (subclass of Problem)
1. Build our Evaluator (don't need to subclass Evaluator)

1.  How do we make it do breadth-first search, depth-first search?  
  1. Which one works better
1.  What are some ideas for a heuristic estimator?


In [13]:
from searchFramework import WorldState
import copy

class EightPuzzleWorldState(WorldState):

    def __init__(self, board):
        self._board = board
        
    def __str__(self):
        return "{" + str(self._board) + "}"
    
    def __eq__(self, other):
        if isinstance(other, EightPuzzleWorldState):
            return self._board == other._board
        else:
            return False

    def __hash__(self):
        return hash(str(self._board))
    
    # NB: every successor state must deep copy the old state!
    
    def successors(self):
        candidates = (self.up(), self.down(), self.left(), self.right())
        return [c for c in candidates if c] 
    
    def up(self):
        bp = self.blankPosition()
        if (bp[0] == 0):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0] -1, bp[1]))
            return((s, "up"))
                   
    def down(self):
        bp = self.blankPosition()
        if (bp[0] == self.boardSize() - 1):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0] + 1, bp[1]))
            return ((s, "down"))   
    def left(self):
        bp = self.blankPosition()
        if (bp[1] == 0):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0], bp[1] - 1))
            return ((s, "left"))
    def right(self):
        bp = self.blankPosition()
        if (bp[1] == self.boardSize() - 1):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0], bp[1] + 1))
            return ((s, "right"))

    def boardSize(self):
        return len(self._board[0])
    
    def swap(self, p1, p2):
        tmp = self._board[p1[0]][p1[1]]
        self._board[p1[0]][p1[1]] = self._board[p2[0]][p2[1]]
        self._board[p2[0]][p2[1]] = tmp
    
    def blankPosition(self):
        for i in range(self.boardSize()):
            for j in range(self.boardSize()):
                   if self._board[i][j] == 0:
                       return (i,j)
        return None
    

In [14]:
from searchFramework import Problem
class EightPuzzleProblem(Problem):
    def __init__(self, initial_board, goal_board = [[1,2,3], [4,5,6], [7,8,0]]):
        self._state = EightPuzzleWorldState(initial_board)
        self._goal_board = goal_board
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state._board == self._goal_board

In [15]:
from searchFramework import Evaluator
def bfsGuesser(state):
    return 0

def bfsCoster(actions):
    return len(actions)

bfsEvaluator = Evaluator(bfsCoster, bfsGuesser)

In [16]:
def dfsGuesser(state):
    return 0

def dfsCoster(actions):
    return -len(actions)

dfsEvaluator = Evaluator(dfsCoster, dfsGuesser)

In [17]:
# Num tiles heuristic

def numTilesCoster(actions):
    return len(actions)

## Cost to goal is number of tiles that are not in the right position

def numTilesEstimator(state):
    #gb = [[1,2,3], [4,5,6], [7,8,0]]
    gb = [[1,2,3], [8,0,4], [7,6,5]]
    est = 0
    for i in range(state.boardSize()):
        for j in range(state.boardSize()):
            if state._board[i][j] != gb[i][j]:
                est += 1
    return est

numTilesEvaluator = Evaluator(numTilesCoster, numTilesEstimator)

In [18]:
# Num tiles heuristic

def numTilesCoster(actions):
    return len(actions)

## Cost to goal is number of tiles that are not in the right position

def tileDistanceEstimator(state):
    #gb = [[1,2,3], [4,5,6], [7,8,0]]
    gb = [[1,2,3], [8,0,4], [7,6,5]]
    est = 0
    for i in range(state.boardSize()):
        for j in range(state.boardSize()):
            est += distanceFrom(state._board, i,j, gb)
    return est

def distanceFrom(board, i, j, goalboard):
    tileValue = board[i][j]
    goalPos = positionOf(tileValue, goalboard)
    return abs(i - goalPos[0]) + abs(j - goalPos[1])

def positionOf(value, board):
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == value:
                return (i,j)
            
tileDistanceEvaluator = Evaluator(numTilesCoster, tileDistanceEstimator)

In [ ]:
easyBoard = [[1,2,3], [4,5,6], [7,0,8]]
easyProblem = EightPuzzleProblem(easyBoard)

In [19]:
from searchFramework import aStarSearch

In [ ]:
print(aStarSearch(easyProblem, bfsEvaluator))

In [ ]:
print(aStarSearch(easyProblem, dfsEvaluator, 1000, 10000))

In [ ]:
print(aStarSearch(easyProblem, numTilesEvaluator))

In [ ]:
harderBoard = [[1, 5,2], [7,4,3], [0, 8, 6]]
harderProblem = EightPuzzleProblem(harderBoard)

In [ ]:
print(aStarSearch(harderProblem, bfsEvaluator))

In [ ]:
print(aStarSearch(harderProblem, dfsEvaluator, None, 5000))

In [ ]:
print(aStarSearch(harderProblem, numTilesEvaluator))

In [ ]:
b = [[2,8,1], [0,4,3], [7,6,5]]
evenHarderProblem = EightPuzzleProblem(b)

In [ ]:
print(aStarSearch(evenHarderProblem, bfsEvaluator))

In [ ]:
print(aStarSearch(evenHarderProblem, numTilesEvaluator))

These examples came from a web page: http://www.d.umn.edu/~jrichar4/8puz.html
but the examples have a different goal state:  
```
[[1,2,3], [8,0,4], [7,6,5]]
```

In [20]:
# Easiest to hardest
b1 = [[1,3,4], [8,6,2], [7,0,5]]
b2 = [[2,8,1], [0,4,3], [7,6,5]]
b3 = [[2,8,1], [4,6,3], [0,7,5]]
b4 = [[5,6,7], [4,0,8], [3,2,1]]

In [22]:
# This redefines the problem to have a different goal state.
# Remember, the evaluators have to redefine the goal too :-(
class DifferentEightPuzzleProblem(Problem):
    def __init__(self, board):
        self._state = EightPuzzleWorldState(board)
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state._board == [[1,2,3], [8,0,4], [7,6,5]]
        

In [23]:
print(aStarSearch(DifferentEightPuzzleProblem(b3), bfsEvaluator, None, 25000))

(['right', 'up', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'down'], (0.90625, 2706, 1087, 1646))


In [24]:
print(aStarSearch(DifferentEightPuzzleProblem(b3), numTilesEvaluator, None, 25000))

(['right', 'up', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'down'], (0.03125, 174, 70, 125))


In [25]:
print(aStarSearch(DifferentEightPuzzleProblem(b3), tileDistanceEvaluator, None, 25000))

(['right', 'up', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'down'], (0.015625, 44, 16, 36))


**Take-away questions**

* Is the problem-solver's ability limited mainly by the fact its heuristic is weak, or is it that the implementation is slow -- it can't expand enough nodes per unit time
* What makes one problem harder than another
* Is the algorithm's performance on BFS and A* search in line with the asymptotic time and space results in the book?